In [1]:
%load_ext autoreload
%autoreload 2

import yaml
yaml.warnings({'YAMLLoadWarning': False})
from fastai import *
from fastai.vision import *
from fastai.callbacks import *
from fastai.distributed import *
from fastai.vision.models.xresnet import *
from fastai.vision.models.unet import DynamicUnet
import PIL.Image
import czifile
from bpho import *
import PIL
PIL.Image.MAX_IMAGE_PIXELS = 99999999999999


In [2]:
data_sources = Path('../data')
live = data_sources/'live'
fixed = data_sources/'fixed'

In [4]:
#!ls {live}/mitotracker/train

In [8]:
tif_files = list(data_sources.glob('**/*.tif'))
czi_files = list(data_sources.glob('**/*.czi'))

def get_tif_size(tif_fn):
    #try:
    shape = {}
    with PIL.Image.open(tif_fn) as img:
        shape['X'] = img.size[0]
        shape['Y'] = img.size[1]
        shape['C'] = 1
        shape['Z'] = img.n_frames
        shape['T'] = 1

    return shape

def get_czi_size(czi_fn):
    try:
        shape = 0
        with czifile.CziFile(czi_fn) as img:
            axes, shape = get_czi_shape_info(img)
            shape = {fld:shape.get(fld, 1) for fld in ['X','Y','C','Z','T']}
    except Exception as ex:
        print(f'{ex} for {czi_fn.stem}')
        shape = None
        
    return shape

def get_info(fn):
    info_proc = {
        '.czi': get_czi_size,
        '.tif': get_tif_size
    }
    info_proc = info_proc.get(fn.suffix, lambda x: None)
    return info_proc(fn)

infos = []
for fn in tif_files + czi_files:
    info = get_info(fn)
    if not info is None:
        info.update({
            'fn': fn,
            'category': fn.parts[-3],
            'fixed_live': fn.parts[-4],
            'train_valid': fn.parts[-2],
            'img_size': max(info['X'], info['Y']),
            'ftype': fn.suffix
        })
        infos.append(info)

In [81]:
mito_files = df.loc[df.category.isin(['mitotracker', 'fixed_cell_mitochondria'])]

In [88]:
mito_files.loc[mito_files.img_size > 3000].fn.apply(lambda x: Path(x).stem[-10:])

233    g_Stitch_2
260    g_Stitch_5
263    _Stitch_10
290    g_Stitch_7
307    g_Stitch_1
507    g_Stitch_9
508    g_Stitch_6
537    g_Stitch_8
552    g_Stitch_3
582    g_Stitch_4
624    _Stitch_11
Name: fn, dtype: object

In [5]:
from train import *

ModuleNotFoundError: No module named 'train'